# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [62]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [63]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bardai,11.6500,7.2333,18.85,19,28,4.53,NG,2024-02-02
1,1,queenstown,-31.8976,26.8753,20.71,79,99,0.47,ZA,2024-02-02
2,2,ribeira grande,38.5167,-28.7000,17.22,94,40,6.17,PT,2024-02-02
3,3,ust-nera,64.5667,143.2000,-29.44,100,100,1.05,RU,2024-02-02
4,4,grytviken,-54.2811,-36.5092,6.34,70,98,7.01,GS,2024-02-02


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [64]:
# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = "City",
    size = "Humidity",
    alpha=0.6,
    frame_width = 800,
    frame_height = 600   
)

# Display the map
# YOUR CODE HERE
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [65]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness
# YOUR CODE HERE
ideal_loc_df = city_data_df[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
# YOUR CODE HERE
ideal_loc_df = ideal_loc_df.dropna()

# Display sample data
# YOUR CODE HERE
ideal_loc_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
81,81,richards bay,-28.7830,32.0377,22.19,84,0,3.12,ZA,2024-02-02
212,212,blackwater,-23.5833,148.8833,25.34,66,0,1.34,AU,2024-02-02
220,220,deland,29.0283,-81.3031,22.21,45,0,3.60,US,2024-02-02
236,236,hermanus,-34.4187,19.2345,24.89,82,0,3.34,ZA,2024-02-02
282,282,gulf shores,30.2460,-87.7008,21.88,58,0,2.24,US,2024-02-02


### Step 3: Create a new DataFrame called `hotel_df`.

In [66]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_loc_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = None

# Display sample data
# YOUR CODE HERE
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
81,richards bay,ZA,-28.7830,32.0377,84,None
212,blackwater,AU,-23.5833,148.8833,66,None
220,deland,US,29.0283,-81.3031,45,None
236,hermanus,ZA,-34.4187,19.2345,82,None
282,gulf shores,US,30.2460,-87.7008,58,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [67]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    long = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
richards bay - nearest hotel: SUN1 Richards Bay
blackwater - nearest hotel: No hotel found
deland - nearest hotel: Courtyard by Marriott DeLand Historic Downtown
hermanus - nearest hotel: Aloe guest house
gulf shores - nearest hotel: Beachside Resort
yeppoon - nearest hotel: Blossom's on Seaspray
las palmas de gran canaria - nearest hotel: Rocktop La Peregrina
merritt island - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
81,richards bay,ZA,-28.7830,32.0377,84,SUN1 Richards Bay
212,blackwater,AU,-23.5833,148.8833,66,No hotel found
220,deland,US,29.0283,-81.3031,45,Courtyard by Marriott DeLand Historic Downtown
236,hermanus,ZA,-34.4187,19.2345,82,Aloe guest house
282,gulf shores,US,30.2460,-87.7008,58,Beachside Resort
294,yeppoon,AU,-23.1333,150.7333,75,Blossom's on Seaspray
377,las palmas de gran canaria,ES,28.0997,-15.4134,48,Rocktop La Peregrina
506,merritt island,US,28.5392,-80.6720,54,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [68]:
# Configure the map plot
# YOUR CODE HERE
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    color = "City",
    tiles = "OSM",
    hover_cols = ["Hotel Name", "Country"],
    size = "Humidity",
    scale = 1.5,
    frame_height = 600,
    frame_width = 800,
    alpha = 0.6
)

# Display the map
# YOUR CODE HERE
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)